# A NEAT Demonstration

NEAT provides a way to define graph machine learning tasks with minimal coding, an uncomplicated interface, and a process created with cloud compute in mind.

This notebook provides a demonstration of how to set up a NEAT configuration file. We define the values for parameters, write them to a YAML file, then pass that file to NEAT to generate graph embeddings.

You're likely reading this notebook while within the NEAT repository. If you haven't installed NEAT yet, please do so now using the next code block.

In [5]:
%cd ..
%pip install .
%cd notebooks/

/home/harry/NEAT
/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Processing /home/harry/NEAT
  Created wheel for neat: filename=neat-0.0.1-py3-none-any.whl size=16819 sha256=2602b38beb324e995db86686823b7c4ce0637a3e730f47ca72d7f131d2b092c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-kxjwin25/wheels/d8/85/56/6759207e7e3026797ec608f40096b52027ce051d31288bb564
Successfully built neat
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.
/home/harry/NEAT/notebooks


## Define graph parameters

For demonstration purposes, we'll use a copy of the [ECTO ontology](https://obofoundry.org/ontology/ecto.html), pre-processed to graph form by [KG-OBO](https://github.com/Knowledge-Graph-Hub/kg-obo).

In [2]:
!wget https://kg-hub.berkeleybop.io/kg-obo/ecto/2022-03-09/ecto_kgx_tsv.tar.gz
!tar xvzf ecto_kgx_tsv.tar.gz

--2022-03-21 14:56:40--  https://kg-hub.berkeleybop.io/kg-obo/ecto/2022-03-09/ecto_kgx_tsv.tar.gz
Resolving kg-hub.berkeleybop.io (kg-hub.berkeleybop.io)... 52.84.52.56, 52.84.52.90, 52.84.52.67, ...
Connecting to kg-hub.berkeleybop.io (kg-hub.berkeleybop.io)|52.84.52.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2753731 (2.6M) [plain/text]
Saving to: ‘ecto_kgx_tsv.tar.gz’

ecto_kgx_tsv.tar.gz 100%[===================>]   2.63M  12.9MB/s    in 0.2s    

2022-03-21 14:56:40 (12.9 MB/s) - ‘ecto_kgx_tsv.tar.gz’ saved [2753731/2753731]

ecto_kgx_tsv_nodes.tsv
ecto_kgx_tsv_edges.tsv


Now define the following graph parameters or just use the default values.

In [6]:
directed = False # Yes, this is technically a directed network, but we'll treat it as undirected
node_path = "ecto_kgx_tsv_nodes.tsv" # Our positive training nodes
edge_path = "ecto_kgx_tsv_edges.tsv" # Our positive training edges - note that this will change shortly

We want to have a positive validation set, too. Let's generate a ~20% holdout set based on the edge file just downloaded.

In [7]:
import random

train_edge_path = (edge_path.split("."))[0] + "_train.tsv"
pos_valid_edge_path = (edge_path.split("."))[0] + "_validate.tsv"

with open(edge_path, 'r') as edge_file:
    header = edge_file.readline()
    with open(train_edge_path, 'w') as train_edge_file:
        train_edge_file.write(header)
        with open (pos_valid_edge_path, 'w') as valid_edge_file:
            valid_edge_file.write(header)
            for line in edge_file:
                if random.randint(1,100) < 20:
                    valid_edge_file.write(line)
                else:
                    train_edge_file.write(line)
                    
edge_path = train_edge_path

Negative data would be nice to have, too. We'll define paths for negative training and negative validation edge lists here, but won't create them until the next section.

In [8]:
neg_train_edge_path = "negative_edges.tsv"
neg_valid_edge_path = "negative_valid_edges.tsv"

## Loading a graph and generating subgraphs

First task: load our training nodes and edges with Ensmallen and get some details about the graph.

In [9]:
from ensmallen import Graph

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [10]:
g= Graph.from_csv(
  directed=directed,
  node_path=node_path,
  edge_path=edge_path,
  verbose=True,
  nodes_column='id',
  node_list_node_types_column='category',
  default_node_type='biolink:NamedThing',
  sources_column='subject',
  destinations_column='object',
  edge_list_edge_types_column='predicate'
)
g

<p>The undirected multigraph graph has 17379 homogeneous nodes and 40218 heterogenous edges.</p><h3>RAM requirements</h3><p>The RAM requirements for the nodes and edges data structures are 4.28MB and 149.82KB respectively.</p><h3>Degree centrality</h3><p>The minimum node degree is 0, the maximum node degree is 4184, the mode degree is 1, the mean degree is 4.63 and the node degree median is 3.</p><p>The nodes with highest degree centrality are: <a href='http://purl.obolibrary.org/obo/chebi#3_STAR' target='_blank' title='Go to Open Biomedical Ontology to get more informations about OBO:chebi#3_STAR'>OBO:chebi#3_STAR</a> (degree 4184 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='http://purl.obolibrary.org/obo/EXO_0000002' target='_blank' title='Go to Exposure ontology to get more informations about ExO:0000002'>ExO:0000002</a> (degree 2286 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='http://purl.obolibrary.org/obo/uberon/core#pheno_slim' target='_blank' title='Go to Uberon Core to get more informations about UBERON_CORE:pheno_slim'>UBERON_CORE:pheno_slim</a> (degree 521 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='http://purl.obolibrary.org/obo/chebi.owl' target='_blank' title='Go to Open Biomedical Ontology to get more informations about OBO:chebi.owl'>OBO:chebi.owl</a> (degree 395 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>) and <a href='http://purl.obolibrary.org/obo/ECTO_0000163' target='_blank' title='Go to Environmental conditions, treatments and exposures ontology to get more informations about ECTO:0000163'>ECTO:0000163</a> (degree 368 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>).</p><h3>Disconnected nodes</h3><p>Disconnected nodes are nodes that are not connected to any other node.The graph contains 998 disconnected nodes.</p><h4>Singleton nodes</h4><p>Singleton nodes are nodes with no edge to other nodes nor selfloops. The graph contains 997 singleton nodes, which are <a href='http://purl.obolibrary.org/obo/NBO_0020088' target='_blank' title='Go to Neuro Behavior Ontology to get more informations about NBO:0020088'>NBO:0020088</a> (node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='http://purl.obolibrary.org/obo/ECTO_9001612' target='_blank' title='Go to Environmental conditions, treatments and exposures ontology to get more informations about ECTO:9001612'>ECTO:9001612</a> (node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='http://purl.obolibrary.org/obo/FOODON_03460141' target='_blank' title='Go to FOODON to get more informations about FOODON:03460141'>FOODON:03460141</a> (node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='http://purl.obolibrary.org/obo/ENVO_00002135' target='_blank' title='Go to Environment Ontology to get more informations about ENVO:00002135'>ENVO:00002135</a> (node type <a hr

Now let's generate some negative edge sets. The following method will create a new Graph object with the specified number of edges, where each is a negative edge in the original graph. It doesn't impact nodes at all, so we remove the now-disconnected nodes for the sake of cleanliness.

In [11]:
negative_graph = g.sample_negatives(g.get_edges_number()) # Just as many negative examples as positive examples
negative_graph = negative_graph.drop_disconnected_nodes()
negative_graph

<h2>Negative Graph</h2><p>The undirected graph Negative Graph has 17205 homogeneous nodes and 40218 edges.</p><h3>RAM requirements</h3><p>The RAM requirements for the nodes and edges data structures are 4.25MB and 149.66KB respectively.</p><h3>Degree centrality</h3><p>The minimum node degree is 1, the maximum node degree is 15, the mode degree is 4, the mean degree is 4.67 and the node degree median is 4.</p><p>The nodes with highest degree centrality are: <a href='http://purl.obolibrary.org/obo/UBERON_0000015' target='_blank' title='Go to Uberon multi-species anatomy ontology to get more informations about UBERON:0000015'>UBERON:0000015</a> (degree 15 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='http://purl.obolibrary.org/obo/RO_0002388' target='_blank' title='Go to Relation Ontology to get more informations about RO:0002388'>RO:0002388</a> (degree 14 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='http://www.snomedbrowser.com/Codes/Details/272626006' target='_blank' title='Go to website to get more informations about http:&#x2f;&#x2f;www.snomedbrowser.com&#x2f;Codes&#x2f;Details&#x2f;272626006'>http:&#x2f;&#x2f;www.snomedbrowser.com&#x2f;Codes&#x2f;Details&#x2f;272626006</a> (degree 14 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>), <a href='https://www.ebi.ac.uk/chebi/searchId.do?chebiId=CHEBI:29258' target='_blank' title='Go to CHEBI to get more informations about CHEBI:29258'>CHEBI:29258</a> (degree 14 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>) and <a href='http://purl.obolibrary.org/obo/ECTO_9000450' target='_blank' title='Go to Environmental conditions, treatments and exposures ontology to get more informations about ECTO:9000450'>ECTO:9000450</a> (degree 14 and node type <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>).</p><h3>Node types</h3><p>The graph has a single node type, which is <a href='https://biolink.github.io/biolink-model/docs/NamedThing.html' target='_blank' title='Go to BioLink to get more informations about biolink:NamedThing'>biolink:NamedThing</a>. Note that this means that all nodes have the same node type, that is, all nodes are homogeneous..</p><h4>RAM requirements</h4><p>The RAM requirements for the node types data structure is 860.69KB.</p>

Now we'll get Ensmallen to generate two different holdout graphs for us, write out those edge files, and continue with defining the NEAT configuration.

In [12]:
neg_train_edge_graph, neg_valid_edge_graph = negative_graph.random_holdout(train_size=0.8)
neg_train_edge_graph.dump_edges(neg_train_edge_path, edges_type_column='predicate')
neg_valid_edge_graph.dump_edges(neg_valid_edge_path, edges_type_column='predicate')

ValueError: Train rate must be strictly between 0 and 1.

## Define embedding parameters

These parameters are quite simple for purposes of the demonstration.

In [21]:
embedding_file_name = "demo_embeddings.tsv"
embedding_history_file_name = "embedding_history.json"
node_embedding_method_name = "CBOW" # one of 'CBOW', 'GloVe', 'SkipGram', 'Siamese', 'TransE', 'SimplE', 'TransH', 'TransR'
walk_length = 10 # typically 100 or so
batch_size = 128 # typically 512? or more
window_size = 4
iterations = 5 # typically 20 or more

## Define classifier parameters

Here, we define a single classifier, but NEAT will accept a list of multiple classifier types.

We're going to set it up as if we were building a link prediction model. NEAT would prefer to have positive *and* negative training/validation data for this. Conveniently, we've provided paths to those above. 

In [22]:
edge_method = "Average" # one of EdgeTransformer.methods: Hadamard, Sum, Average, L1, AbsoluteL1, L2, or alternatively a lambda
classifier_type = "Logistic Regression"
classifier_model_outfile = "model_lr_demo"
classifier_model_type = "sklearn.linear_model.LogisticRegression"
classifier_model_random_state = 42
classifier_model_max_iter = 1000

## Define output parameters

We specify a local output path here, but NEAT can also upload to S3, given a bucket name and directory.

In [23]:
output_directory = "./"

config_filename = "demonstrate.yaml"

## Wrap it all up

In [24]:
outstring = f"""
graph_data:
  graph:
    directed: {directed}
    node_path: {node_path}
    edge_path: {edge_path}
    verbose: True
    nodes_column: 'id'
    node_list_node_types_column: 'category'
    default_node_type: 'biolink:NamedThing'
    sources_column: 'subject'
    destinations_column: 'object'
    default_edge_type: 'biolink:related_to'
  pos_validation:
    edge_path: {pos_valid_edge_path}
  neg_training:
    edge_path: {neg_train_edge_path}
  neg_validation:
    edge_path: {neg_valid_edge_path}

embeddings:
  embedding_file_name: {embedding_file_name}
  embedding_history_file_name: {embedding_history_file_name}
  node_embedding_params:
      node_embedding_method_name: {node_embedding_method_name}
      walk_length: {walk_length}
      batch_size: {batch_size}
      window_size: {window_size}
      return_weight: 1.0
      explore_weight: 1.0
      iterations: {iterations}
      use_mirrored_strategy: False

  tsne:
    tsne_file_name: tsne.png

classifier:
  edge_method: {edge_method}
  classifiers:
    - type: {classifier_type}
      model:
        outfile: {classifier_model_outfile}
        type: {classifier_model_type}
        parameters:
          random_state: {classifier_model_random_state}
          max_iter: {classifier_model_max_iter}

output_directory: {output_directory}
"""
print(outstring)


graph_data:
  graph:
    directed: False
    node_path: ecto_kgx_tsv_nodes.tsv
    edge_path: ecto_kgx_tsv_edges.tsv
    verbose: True
    nodes_column: 'id'
    node_list_node_types_column: 'category'
    default_node_type: 'biolink:NamedThing'
    sources_column: 'subject'
    destinations_column: 'object'
    default_edge_type: 'biolink:related_to'
  pos_validation:
    edge_path: ecto_kgx_tsv_edges.tsv
  neg_training:
    edge_path: negative_edges.tsv
  neg_validation:
    edge_path: negative_valid_edges.tsv

embeddings:
  embedding_file_name: demo_embeddings.tsv
  embedding_history_file_name: embedding_history.json
  node_embedding_params:
      node_embedding_method_name: CBOW
      walk_length: 10
      batch_size: 128
      window_size: 4
      return_weight: 1.0
      explore_weight: 1.0
      iterations: 5
      use_mirrored_strategy: False

  tsne:
    tsne_file_name: tsne.png

classifier:
  edge_method: Average
  classifiers:
    - type: Logistic Regression
      model:
  

In [25]:
with open(config_filename, "w") as outfile:
    outfile.write(outstring)

In [26]:
!neat run --config $config_filename

Traceback (most recent call last):
  File "/home/harry/neat-env/bin/neat", line 8, in <module>
    sys.exit(cli())
  File "/home/harry/neat-env/lib/python3.8/site-packages/click/core.py", line 1128, in __call__
    return self.main(*args, **kwargs)
  File "/home/harry/neat-env/lib/python3.8/site-packages/click/core.py", line 1053, in main
    rv = self.invoke(ctx)
  File "/home/harry/neat-env/lib/python3.8/site-packages/click/core.py", line 1659, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/harry/neat-env/lib/python3.8/site-packages/click/core.py", line 1395, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/home/harry/neat-env/lib/python3.8/site-packages/click/core.py", line 754, in invoke
    return __callback(*args, **kwargs)
  File "/home/harry/neat-env/lib/python3.8/site-packages/neat/cli.py", line 41, in run
    if not pre_run_checks(yhelp=yhelp):
  File "/home/harry/neat-env/lib/python3.8/site-packages/neat/pre_run_check

In [ ]:
from IPython.display import Image
Image(filename='tsne.png') 